Próba implementacji metod zastosowanych przez dr. Kontka w artykule 

["**Weryfikacja wyniku drugiej tury wyborów prezydenckich w Polsce w 2025 roku: Przeliczenie głosów z użyciem przestrzennie grupowanej metody MAD**"](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=5296441)

dla wyników wyborów prezydenckich z 2025 ale przy grupowaniu komisji losowo.

Celem eksperymentu jest sprawdzenie, czy wykrywalność tzw. anomalii wyborczych (wg metody MAD) zależy faktycznie od przestrzennego podobieństwa komisji, czy też wynika jedynie z naturalnego rozrzutu procentowego poparcia, niezależnie od kontekstu.

Jeśli metoda Konteka jest trafna i opiera się na założeniu, że komisje w pobliżu mają podobne wyniki, to losowe grupowanie (bez związku przestrzennego) powinno znacząco zmniejszyć liczbę wykrywanych anomalii.
Jeśli jednak liczba anomalii pozostanie podobna – jak sugerują wstępne symulacje – może to wskazywać, że metoda nie wykrywa rzeczywistych odstępstw, lecz generuje nadmiarowe wyniki w wyniku statystycznych fluktuacji.

In [6]:
import pandas as pd
import numpy as np

from utilities import presidential_data, clustering, methods

### load presidential both rouds - official PKW data

In [23]:
year = "2025"
df_2025_r1 = presidential_data.get_df(year, "1")
df_2025_r2 = presidential_data.get_df(year, "2")

### IMPLEMENTACJA METOD ZAPROPONOWANYCH PRZEZ DR KONTKA

2.3. Wykrywanie wartości odstających

Główna innowacja analityczna niniejszego badania polega na oszacowaniu potencjalnego wpływu
anormalnych komisji wyborczych na poziomie ogólnokrajowym. Aby to osiągnąć, w pierwszej
kolejności zidentyfikowano wartości odstające w czterech kategoriach nieprawidłowości:

In [24]:
cand_A = "trzaskowski"
cand_B = "nawrocki"

df_2025 = presidential_data.join_both_rounds(cand_A, cand_B, df_2025_r1, df_2025_r2)

In [25]:
df_2025.head()

,teryt_gmina,polling_station_id,trzaskowski_r1,nawrocki_r1,postal_code,trzaskowski_r2,nawrocki_r2
0,20101,1,361,287,59-700,596,582
1,20101,2,381,228,59-700,517,386
2,20101,3,356,241,59-700,556,437
3,20101,4,390,217,59-700,567,408
4,20101,5,343,202,59-700,518,332


In [26]:
def add_random_buckets_deterministic(df, n, k_min, k_max, random_state=None):
    """
    Deterministycznie i losowo przypisuje komisje do n grup (bucketów),
    z kontrolą rozmiarów i zachowaniem powtarzalności.

    Sortuje dane po teryt_gmina i polling_station_id, a następnie je przetasowuje
    (przy użyciu ziarna random_state), by uniknąć tworzenia sztucznie podobnych grup.

    Returns:
        pd.DataFrame z kolumną 'bucket'
    """
    N = len(df)
    if N < n * k_min or N > n * k_max:
        raise ValueError("Constraints cannot be satisfied with given n, k_min, and k_max.")

    sizes = np.full(n, k_min)
    remaining = N - sizes.sum()
    increments = np.zeros(n, dtype=int)

    rng = np.random.default_rng(random_state)
    while remaining > 0:
        idxs = np.where(sizes + increments < k_max)[0]
        idx = rng.choice(idxs)
        increments[idx] += 1
        remaining -= 1

    final_sizes = sizes + increments

    # Uporządkowana baza – dla powtarzalności
    sorted_df = df.sort_values(by=["teryt_gmina", "polling_station_id"]).reset_index(drop=True)

    # Pełny shuffle – dla losowości grup
    shuffled_df = sorted_df.sample(frac=1, random_state=random_state).reset_index(drop=True)

    bucket_labels = np.repeat(np.arange(n), final_sizes)
    shuffled_df["bucket"] = bucket_labels

    return shuffled_df


In [27]:
# losowanie grupowe, ale deterministyczne (dla celów re-produkcji analizy)
df = df_2025.copy()

In [47]:
df = add_random_buckets_deterministic(
    df,
    n=2208,           # liczba grup
    k_min=10,         # minimalna liczba komisji w grupie
    k_max=16,         # maksymalna liczba komisji w grupie
    random_state=42   # dla powtarzalności
)
df["postal_clean"] = df["postal_code"].astype(str).str.replace("-", "")

In [48]:
print("Liczba komisji: ", len(df))

Liczba komisji:  31627


### 1. pop_outlier: Nadmierne poparcie dla kandydata A (względem mediany w ramach lokalnej grupy)

In [49]:
df = methods.add_anomaly_1(df, cand_A, cand_B, new_col_name="k_score_1")

In [50]:
methods.check_k_thresholds(df, cand_A, cand_B, "_k_score_1")

k > 2.0
trzaskowski: 6234
nawrocki: 3771
---
k > 2.5
trzaskowski: 5081
nawrocki: 2608
---
k > 3.0
trzaskowski: 4198
nawrocki: 1835
---


Wyniki:

Dla k=2, takich komisji, w których "za duże" poparcie ma Trzaskowski jest 6234 a Nawrocki 3771

### 2. growth_outlier: Nadmierny względny wzrost poparcia dla kandydata A między pierwszą a drugą turą, w porównaniu do odpowiedniego wzrostu poparcia dla kandydata B w tej samej grupie lokalnej;

In [51]:
df = methods.add_anomaly_2(df, cand_A, cand_B, new_col_name="k_score_2")

In [52]:
methods.check_k_thresholds(df, cand_A, cand_B, "_k_score_2")

k > 2.0
trzaskowski: 4119
nawrocki: 3313
---
k > 2.5
trzaskowski: 3162
nawrocki: 2344
---
k > 3.0
trzaskowski: 2478
nawrocki: 1705
---


### 3. Komisje, w których kand A uzyskał więcej głosów niż kand B w drugiej turze, mimo że mediana wyników w grupie wskazywała na przewagę kand B;

In [53]:
df = methods.add_anomaly_3(df, cand_A, cand_B, new_col_name="flip")

In [54]:
# na korzyść kandydata A, czyli większą medianę miał B, a więcej głosów dostał A.
cand_A, sum(df['higher_median_' + cand_B] & (df[cand_A + '_r2'] > df[cand_B + '_r2']))

('trzaskowski', 9290)

In [55]:
cand_B, sum(df['higher_median_' + cand_A] & (df[cand_B + '_r2'] > df[cand_A + '_r2']))

('nawrocki', 2479)

In [56]:
df = df.drop(
    columns=[
        f"higher_median_{cand_A}",
        f"higher_median_{cand_B}",
    ]
)

**WYNIKI**:

W grupach, w których większą medianę miał Nawrocki, było 2479 komisji, w których wyższy wynik uzyskał Trzaskowski.

W grupach, w których większą medianę miał Trzaskowski, było 9290 komisji, w których wyższy wyniki uzyskał Nawrocki.

### 4. Kandydat otrzymał mniej głosów w drugiej turze niż w pierwszej

In [57]:
cand_A, sum(df[cand_A + '_r2']<df[cand_A + '_r1'])

('trzaskowski', 128)

W 128 komisjach Trzaskowski uzyskał mniej głosów w drugiej turze niż w pierwszej.

In [58]:
cand_B, sum(df[cand_B + '_r2']<df[cand_B + '_r1'])

('nawrocki', 113)

W 113 komisjach Nawrocki uzyskał mniej głosów w drugiej turze niż w pierwszej.


Przykładowe anomalie na korzyść Trzaskowskiego:

In [59]:
df[df[cand_B + '_r2'] < df[cand_B + '_r1']].sort_values(by=cand_B + '_r1', ascending=False).head()

,teryt_gmina,polling_station_id,trzaskowski_r1,nawrocki_r1,postal_code,trzaskowski_r2,nawrocki_r2,bucket,trzaskowski_k_score_1,nawrocki_k_score_1,trzaskowski_k_score_2,nawrocki_k_score_2,trzaskowski_flip,nawrocki_flip,trzaskowski_more_votes,nawrocki_more_votes,postal_clean
20526,140706,1,105,285,26-910,467,193,1430,0.953216,-1.495495,28.438218,-28.438218,True,False,True,False,26910
11288,261207,4,143,224,28-200,360,209,784,0.984026,-0.635945,7.750516,-7.750516,True,False,True,False,28200
5164,60903,4,89,174,23-100,260,163,359,0.686099,-0.823045,7.120588,-7.120588,True,False,True,False,23100
12114,260101,34,172,129,28-100,148,111,844,-0.636000,-1.951020,0.157427,-0.157427,True,False,True,True,28100
13214,40102,9,120,105,87-720,164,85,921,-1.916031,-1.919298,6.545199,-6.545199,False,False,True,False,87720


In [60]:
# Anomalie na korzysc
df[cand_A + '_more_votes'] = df[cand_B + '_r2']<df[cand_B + '_r1']
df[cand_B + '_more_votes'] = df[cand_A + '_r2']<df[cand_A + '_r1']

## Wyniki

In [61]:
# w zależności od wielkości k
k = 2

# deduplikacja, czyli jak wystąpiła więcej niz jedna anomalia, to liczymy taką komisję raz

### Na korzyść Nawrockiego

In [62]:
cand = "nawrocki"
opponent = "trzaskowski"
outliers_df = methods.generate_candidate_outliers(df, cand, opponent, k)

### Liczba występujących anomalii. Czyli w ilu komisjach wystąpiła np. anomalia "pop outlier"

In [63]:
bool_cols = ["pop_outlier", "growth_outlier", "flip", "more_votes"]
print(outliers_df[bool_cols].sum())

pop_outlier       3771
growth_outlier    3313
flip              2479
more_votes         128
dtype: int64


### Liczba komisji, w których wystąpiło minimum N anomalii, czyli minimum 1 (8910) albo wszystkie cztery anomalie (1)

In [64]:
true_counts = outliers_df[bool_cols].sum(axis=1)

# Convert to plain ints
count_distribution = {
    ">=1": int((true_counts >= 1).sum()),
    ">=2": int((true_counts >= 2).sum()),
    ">=3": int((true_counts >= 3).sum()),
    ">=4": int((true_counts >= 4).sum())
}

# Print row by row
for key, value in count_distribution.items():
    print(f"{key}: {value}")

>=1: 8910
>=2: 759
>=3: 21
>=4: 1


### Próba odwzorowania tabelki z wynikami

In [65]:
# suma głosów. jeżeli w komisji występuje więcej niż jedna anomalia, sumuj jej głosy tylko raz
outliers_df["anomalies"] = outliers_df.apply(methods.assign_top_anomaly, axis=1)

In [66]:
outliers_df[cand + "_median_r2"] = outliers_df.groupby("bucket")[cand + "_r2"].transform("median")
outliers_df[opponent + "_median_r2"] = outliers_df.groupby("bucket")[opponent + "_r2"].transform("median")

In [67]:
outliers_df = methods.add_median_corrected_votes(outliers_df, cand, opponent)

In [68]:
summary_df = methods.summarize_by_anomaly(outliers_df, cand, opponent)
summary_df

,flaga,liczba,głosy nawrocki,głosy trzaskowski,różnica przed,głosy nawrocki po,głosy trzaskowski po,różnica po,zmiana
0,pop_outlier,3254,2154046,1746691,407355,2321667,1579070,742597,335242
1,growth_outlier,3061,716473,1075849,-359376,1032451,759871,272580,631956
2,flip,2467,810847,462299,348548,594729,678417,-83688,-432236
3,more_votes,128,7028,3441,3587,5867,4602,1265,-2322
4,łącznie,8910,3688394,3288280,400114,3954714,3021960,932754,532640


### Na korzyść Trzaskowskiego

In [69]:
cand = "trzaskowski"
opponent = "nawrocki"
outliers_df = methods.generate_candidate_outliers(df, cand, opponent, k)

### Liczba występujących anomalii. Czyli w ilu komisjach wystąpiła np. anomalia "pop outlier"

In [70]:
bool_cols = ["pop_outlier", "growth_outlier", "flip", "more_votes"]
print(outliers_df[bool_cols].sum())

pop_outlier       6234
growth_outlier    4119
flip              9290
more_votes         113
dtype: int64


### Liczba komisji, w których wystąpiło minimum N anomalii, czyli minimum 1 (14642) albo wszystkie cztery anomalie (0)

In [71]:
true_counts = outliers_df[bool_cols].sum(axis=1)

# Convert to plain ints
count_distribution = {
    ">=1": int((true_counts >= 1).sum()),
    ">=2": int((true_counts >= 2).sum()),
    ">=3": int((true_counts >= 3).sum()),
    ">=4": int((true_counts >= 4).sum())
}

# Print row by row
for key, value in count_distribution.items():
    print(f"{key}: {value}")

>=1: 14642
>=2: 5009
>=3: 105
>=4: 0


In [72]:
outliers_df["anomalies"] = outliers_df.apply(methods.assign_top_anomaly, axis=1)

In [73]:
outliers_df[cand + "_median_r2"] = outliers_df.groupby("bucket")[cand + "_r2"].transform("median")
outliers_df[opponent + "_median_r2"] = outliers_df.groupby("bucket")[opponent + "_r2"].transform("median")

In [74]:
outliers_df = methods.add_median_corrected_votes(outliers_df, cand, opponent)

In [75]:
summary_df = methods.summarize_by_anomaly(outliers_df, cand, opponent)
summary_df

,flaga,liczba,głosy trzaskowski,głosy nawrocki,różnica przed,głosy trzaskowski po,głosy nawrocki po,różnica po,zmiana
0,pop_outlier,1549,994296,893947,100349,787738,1100505,-312767,-413116
1,growth_outlier,3750,337891,1073873,-735982,579619,832145,-252526,483456
2,flip,9230,5083611,3132434,1951177,3272054,4943991,-1671937,-3623114
3,more_votes,113,3533,2289,1244,2428,3394,-966,-2210
4,łącznie,14642,6419331,5102543,1316788,4641839,6880035,-2238196,-3554984
